## Classify Emails beyond spams - Data Preprocessing

### This notebook deals with data preprocessing. A csv file is exported which is used in the other files as input data. this fine need to get excecuted first as it generates the input file for other notebooks.

### Final Project - Riti Chakraborty

In [1]:
#Riti Chakraborty

#importing the required libraries
import pandas as pd
import numpy as np
from numpy import nan

#For random seed
import random

#To handle warnings
import warnings
warnings.filterwarnings('ignore')



#For flattening lists
from itertools import chain




#library for tf-idf vector and other similarities measure
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



#For using Regular expression
import re

#For Handling Strings
import string






In [2]:
### Reading the .csv file generated after 

In [3]:
#Reading Email_train Data.
Emails = pd.read_csv('../input_data/TRAININGEMAILS.csv')

#Adding the label column 0-Spam and 1- Ham
label = pd.read_csv('../input_data/Label.txt', sep=" ", header=None)
Emails['label'] = label[0]


#Keeping those records for which From Field in not Empty i.e. Removing Empty Emails.
Emails_Notnull= Emails[pd.notnull(Emails['From'])]
print("Total Number of Emails in the Dataset:",Emails_Notnull.shape[0])

#Seperatinbg out the spams
Spams = Emails_Notnull.loc[Emails_Notnull['label']==0]
Hams = Emails_Notnull.loc[Emails_Notnull['label']==1]
Spams.to_csv('../exported_tables/Spams_1.csv')

#Subsetting records with only 20% NA values 
Spams_set = Spams.loc[:, pd.notnull(Spams).sum()>len(Spams)*.8]
print("Total Number of Spams Available:",Spams_set.shape[0])

Total Number of Emails in the Dataset: 4035
Total Number of Spams Available: 1241


In [4]:
#Renaming the required Columns
Spams_set = Spams_set.rename(columns={'Message-id': 'MessageID'})

#Renaming the columns Content type and MIME Version
Spams_set = Spams_set.rename(columns={'Content-type': 'ContentType'})
Spams_set = Spams_set.rename(columns={'Mime-version': 'MimeVersion'})


#Seperating out values
#Column: MessageID
Spams_set['LocalPart'] = Spams_set.MessageID.str.split('@').str.get(0)
Spams_set['Domain'] = Spams_set.MessageID.str.split('@').str.get(1)

#Column: From
Spams_set['From1'] = Spams_set.From.str.split('@').str.get(0)
Spams_set['From2'] = Spams_set.From.str.split('@').str.get(1)    


#Column: Received
Spams_set['Receivedfrom'] = Spams_set.Received.str.split('by').str.get(0)
Spams_set['Receivedby'] = Spams_set.Received.str.split('by').str.get(1)    


#Column: Date
Spams_set['Date']=pd.to_datetime(Spams_set['Date'], errors = 'coerce')

#Column: ReturnPath
Spams_set = Spams_set.rename(columns={'Return-path': 'Returnpath'})
Spams_set['Returnpath1'] = Spams_set.Returnpath.str.split('@').str.get(0)
Spams_set['Returnpath2'] = Spams_set.Returnpath.str.split('@').str.get(1)    

#Column: To
Spams_set['To1'] = Spams_set.To.str.split('@').str.get(0)
Spams_set['To2'] = Spams_set.To.str.split('@').str.get(1)    

#Column: Delivered To
Spams_set = Spams_set.rename(columns={'Delivered-to': 'Deliveredto'})
Spams_set['DeliveredTo1'] = Spams_set.Deliveredto.str.split('@').str.get(0)
Spams_set['DeliveredTo2'] = Spams_set.Deliveredto.str.split('@').str.get(1)    

#Extracting rows where date is not null
Spams_set = Spams_set[pd.notnull(Spams_set['Date'])]


#Extracting various different features from date
Spams_set["Date"]=pd.to_datetime(Spams_set["Date"])
Spams_set["Year"]=Spams_set["Date"].dt.year
Spams_set["Month"]=Spams_set["Date"].dt.strftime("%B")
Spams_set["Day"]=Spams_set["Date"].dt.strftime("%A")
Spams_set["Hour"]=Spams_set["Date"].dt.hour

Spams_set['Year'] = 'Year' + Spams_set['Year'].astype(str)
Spams_set['Hour'] = 'Hour' + Spams_set['Hour'].astype(str)


In [5]:
#Rechecking for empty rows
Spams_set.drop(Spams_set.query("Subject == ''| ContentType == ''|  MimeVersion == '' | From == '' | MessageID == '' | DeliveredTo1 == '' | DeliveredTo2 == '' | label == '' | LocalPart == '' | Domain == '' | From1 == '' | From2 == '' | Receivedfrom == '' | Receivedby == '' | Returnpath1 == '' | Returnpath2 == '' | To1 == '' | To2 == '' | DeliveredTo1 == '' | DeliveredTo2 == '' | Year == '' | Month == '' | Day == '' |  Hour == ''").index, inplace=True)

In [6]:
#Transforming IP address
rec_column = Spams_set['Received'].tolist()
ip_1=[]
for i in rec_column:
    ip =  re.findall(r'\[(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})\]', i)
    if len(ip) > 1:
        ip_1.append([ip[0]])
    else:
        ip_1.append(ip)

ip_1df=pd.DataFrame(ip_1)

### Reverse Geocoding -> Extracting the Location name from the IP address 

In [7]:
import pycountry
pycountry.countries.get(alpha_2='CN').name
import geocoder

# country_ip=[]
# for j in ip_1df[0].tolist():
#     if j != None:
#         g = geocoder.ip(j)
#         country_ip.append(g.country)
# #         print(g.country)
#     else:
#         country_ip.append(None)
# pd.DataFrame(country_ip).to_csv("../exported_tables/country_ip.csv")

In [8]:
country_ip1=pd.read_csv("../exported_tables/country_ip.csv")
country_ip2=country_ip1['0'].tolist()
country_ip=[str(s) for s in country_ip2] 


Loc_from_ip=[]
for c_ip in country_ip:
#     print(c_ip)
    if c_ip != 'nan' and c_ip != 'EU':
        Loc_from_ip.append(pycountry.countries.get(alpha_2=c_ip).name)
    elif c_ip == 'EU':
        Loc_from_ip.append("Europe")
    else:
        Loc_from_ip.append(None)


### After Preliminary tests, attributes which didnot contribute in clustering were removed. The attributes mentioned below are the finally chosen ones.

In [9]:
#Subsetting the dataframe extracting the columns which are created to store seperated features.
data_subset = Spams_set[["Subject", "From1", "From2", "Returnpath2", "Month", "Day"]]
loc_ip=pd.DataFrame(Loc_from_ip)
loc_ip = loc_ip.rename(columns={0: 'Location'})

data_subset['Location_1']=pd.Series(loc_ip['Location'])

data_subset.shape
data_subset.to_csv("../exported_tables/data_subset.csv")


